<h1> PyTorch in a nutshell </h1> 

PyTorch is a module built for working with tensors and particularly adapt for building deep learning models in image recognition and language processing. As a python module, it aims to be easy to implement and provide code that is generally very readable.
In this notebook we will follow the tutorial presented on the [PyTorch Page](https://pytorch.org/tutorials/beginner/basics/quickstart_tutorial.html), which mainly focuses on computer vision applications. 

The required modules can be installed on the conda environment with: